# **성능관리_종합실습**

## **(1) 환경준비**

### **1) 라이브러리 불러오기**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.backend import clear_session
from keras.optimizers import Adam
from keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping

from keras.datasets import fashion_mnist

### **2) 함수만들기**

In [ ]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.figure(figsize=(10,6))
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

### **3) 데이터 로딩**

In [ ]:
(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

rd.seed(20)
tr_id = rd.sample(range(60000), 4000)
va_id = rd.sample(range(10000), 1000)

x_train, y_train = x_train[tr_id], y_train[tr_id]
x_val, y_val = x_val[va_id], y_val[va_id]

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### **4) 데이터 둘러보기**

In [ ]:
# 아래 숫자를 바꿔가며 화면에 그려 봅시다.
n = 1
i = y_val[n]

plt.imshow(x_val[n], cmap=plt.cm.binary)
plt.title(f'y : {class_names[i]}')
plt.colorbar()
plt.show()

## **(2) 데이터준비**

### 1) 데이터 2차원으로 펼치기

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

In [ ]:
# 펼치기
x_train = x_train.reshape(4000, -1)
x_val = x_val.reshape(1000, -1)

In [ ]:
x_train.shape, x_val.shape

### 2) Scaling : Min-Max
* 0-255 값으로 되어 있는 데이터를 0-1사이 값으로 변환
* x_train, x_val를 그냥 255로 나누면 됨

In [ ]:
x_train = x_train / 255.
x_val = x_val / 255.

## (3) 모델링

* 히든레이어의 노드를 다음과 같이 구성합니다.
    * 128, 64, 32, 16, 10
* epoch : 50 ~
* learning_rate : 0.01 ~ 0.0001


### 1) 모델1 : 과적합 모델
* 위에서 제시한 구조대로 모델을 설계하고 학습합니다.

#### ① 모델설계

In [ ]:
# feature 수
nfeatures = x_train.shape[1]

# 메모리정리
clear_session()

# 모델설계
model1 = Sequential([Input(shape = (nfeatures,)),
                     Dense(128, activation = 'relu'),
                    Dense(64, activation = 'relu'),
                    Dense(32, activation = 'relu'),
                    Dense(16, activation = 'relu'),
                    Dense(10, activation = 'softmax')])

# 모델요약
model1.summary()

# 컴파일
model1.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy')

#### ② 학습

In [ ]:
# 학습
hist = model1.fit(x_train, y_train, epochs = 100, validation_split=0.2).history

In [ ]:
# 학습곡선
dl_history_plot(hist)

#### ③ 예측 및 검증평가

In [ ]:
# 예측
pred1 = model1.predict(x_val)
pred1_1 = np.argmax(pred1, axis = 1)

In [ ]:
# 평가
print(confusion_matrix(y_val, pred1_1))
print('-' * 55)
print(f'accuracy : {accuracy_score(y_val, pred1_1)}')
print('-' * 55)
print(classification_report(y_val, pred1_1, target_names = class_names))

### 2) early stopping
* 모델1에 대해서, early stopping으로 과적합을 예방해 봅시다.

#### ① 모델설계

In [ ]:
# feature 수
nfeatures = x_train.shape[1]

# 메모리정리
clear_session()

# 모델설계
model2 = Sequential([Input(shape = (nfeatures,)),
                     Dense(128, activation = 'relu'),
                    Dense(64, activation = 'relu'),
                    Dense(32, activation = 'relu'),
                    Dense(16, activation = 'relu'),
                    Dense(10, activation = 'softmax')])

# 모델요약
model2.summary()

# 컴파일
model2.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy')

#### ② 학습

In [ ]:
# EarlyStopping 설정 ------------------
es = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, patience = 5)

hist = model2.fit(x_train, y_train, epochs = 100, validation_split=0.2, callbacks = [es]).history

In [ ]:
# 학습곡선
dl_history_plot(hist)

#### ③ 예측 및 검증평가

In [ ]:
# 예측
pred2 = model2.predict(x_val)
pred2_1 = np.argmax(pred2, axis = 1)

In [ ]:
# 평가
print(confusion_matrix(y_val, pred2_1))
print('-' * 55)
print(f'accuracy : {accuracy_score(y_val, pred2_1)}')
print('-' * 55)
print(classification_report(y_val, pred2_1, target_names = class_names))

### 3) Dropout
* 모델1에 대해서, Dropout으로 과적합을 방지해 봅시다.

#### ① 모델설계

In [ ]:
# feature 수
nfeatures = x_train.shape[1]

# 메모리정리
clear_session()

# 모델설계
model3 = Sequential([Input(shape = (nfeatures,)),
                     Dense(128, activation = 'relu'),
                     Dropout(0.3),
                     Dense(64, activation = 'relu'),
                     Dropout(0.3),
                     Dense(32, activation = 'relu'),
                     Dropout(0.3),
                     Dense(16, activation = 'relu'),
                     Dropout(0.3),
                     Dense(10, activation = 'softmax')])

# 모델요약
model3.summary()

# 컴파일
model3.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy')

#### ② 학습

In [ ]:
# 학습
hist = model3.fit(x_train, y_train, epochs = 100, validation_split=0.2).history

In [ ]:
# 학습곡선
dl_history_plot(hist)

#### ③ 예측 및 검증평가

In [ ]:
# 예측
pred3 = model3.predict(x_val)
pred3_1 = np.argmax(pred3, axis = 1)

In [ ]:
# 평가
print(confusion_matrix(y_val, pred3_1))
print('-' * 55)
print(f'accuracy : {accuracy_score(y_val, pred3_1)}')
print('-' * 55)
print(classification_report(y_val, pred3_1, target_names = class_names))